# PepperOpenDayDemo

## Environment Setup

In [1]:
# built-in imports
# !pip install requests
# !pip install speechrecognition==3.8.1
# !pip install paramiko
# !pip install pandas
# !pip install beautifulsoup4
# !pip install pocketsphinx

In [1]:
### Naoqi Related Imports ###
import naoqi
from naoqi import qi
from naoqi import ALProxy

### System Related Imports ###
import fileinput, os, shutil, sys, random, threading, time

### Flask Related Imports ###
import json, requests

### Speech Recognition Imports ###
import pandas as pd
import paramiko
import speech_recognition as sr

/usr/local/lib/python2.7/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography.hazmat.backends import default_backend


In [2]:
from prompts import *
import prompts
from fileTransfer import *
import fileTransfer as ft
from displayGeneration import *
import displayGeneration as dg
from interactiveControls import *
import interactiveControls as ic
from idle import *
import idle as idl
from humanInteraction import *
import humanInteraction as hi


# ### Custom File Handing Imports ###
# from fileTransfer import *

# ### Custom Idle Behaviour Imports ###
# from idle import *

# ### Custom Page Display Imports ###
# from displayGeneration import *

# ### Custom Interaction, Behaviour and Display Imports ###
# from interactiveControls import *

# ### Custom Hard-Coded Prompt Imports ###
# from prompts import *

# ### Main Interaction Imports ###
# from humanInteraction import *

## Constants

In [3]:
### Networking Constants ###
GPT_HOST = "10.104.23.208"
PEPPER_HOST = "10.104.23.185"
GPT_PORT = 8891
PEPPER_PORT = 9559

### Audio Processing Constants ###
PEPPER_RECORDINGS_PATH = "/home/nao/microphones/recording.wav"
SERVER_RECORDINGS_PATH = "recordings/recording.wav"

### Display Generation Constants ###
FILE_NAME_TEMP = "display.html"
PEPPER_HTML_PATH = "/home/nao/.local/share/PackageManager/apps/robot-page/html/"  # page.html"
PEPPER_PAGE_LANDING = PEPPER_HTML_PATH + "page.html" # '/home/nao/.local/share/PackageManager/apps/robot-page/html/page.html'
PEPPER_QR_LANDING = PEPPER_HTML_PATH + "webfiles/qr.png"  # switched slash direction
PEPPER_IMG_LANDING = PEPPER_HTML_PATH + "webfiles/img.png"  # switched slash direction
TEXT_BY_ID_PATH = "textbyID.csv"

## MAIN

In [5]:
global idle
global responsesPipeline
global eyes

idle = idling()
eyes = eyesController()

try:
    track_head()
    repeat = True

    while True:
        # Empty sentences list
        responsesPipeline = []
        
        # Idle behaviors to keep peppers temperature low
        idle.start_idle_behavior()

        # trigger smart listen
        detect()
        idle.stop()
        
        ## If speech detected record up to 15 seconds of audio
        query = listen()
        repeat = queryCourseCodes(query, responsesPipeline, eyes)
        
        # get course codes relevant to this
        while repeat:
            query = listen()
            repeat = queryCourseCodes(query, responsesPipeline, eyes)
            
        #ask if they need specific details about a course
        repeat = True
        rcount = 0
        while repeat:
            query = listen()
            repeat = querySpecificCourse(query, responsesPipeline, eyes, rcount)
            rcount += 1

        # get gpt response
        # think(query, responsesPipeline, eyes)
        
except Exception as e:
    print(e)
finally:
    stopListening()
    return_to_default_pos()
    idle = idling()
    idle.start_idle_behavior()
    print('Goodbye')

copyfile successful
Sent to Pepper
0.466121554375
Detect complete
Recording is starting...
Timer is reached
Recording is stopping...
Sent from Pepper
('Listen complete, This is what I heard: ', u'courses related to artificial intelligence and designs')
copyfile successful
Sent to Pepper
Recording is starting...
Timer is reached
Recording is stopping...
Sent from Pepper
('Listen complete, This is what I heard: ', u'master of applied artificial intelligence professional')
copyfile successful
Sent to Pepper
threads can only be started once
Goodbye


In [ ]:

# ic.showWhichPage("loading")
# ic.showWhichPage("Cacc")
# ic.showWhichPage("Cour")
# ic.showWhichPage("Cstu")
# ic.showWhichPage("Cwel")
# ic.showWhichPage("Club")

dg.generateBasicQRPage("D310")
showPage()

In [ ]:

ic.showWhichPage("loading")


copyfile successful
Sent to Pepper
